In [ ]:
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, QuantumRegister, ClassicalRegister, opflow
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit_aer import AerSimulator
import numpy as np
import math 
import json
from qiskit.opflow import I, X, Y, Z, MatrixEvolution, PauliTrotterEvolution
from qiskit.circuit import Parameter
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt
from qiskit.providers.aer import AerSimulator
from scipy.linalg import expm, sinm, cosm
import cmath
import scipy.integrate as integrate
import scipy.special as special
import seaborn as sns
from numpy import random, linalg as LA
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import MaxNLocator
from scipy.ndimage.filters import gaussian_filter
import plotly.express as px
import pandas as pd
from qiskit.tools.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
from qiskit.test.mock import FakeJakarta
import plotly.graph_objects as go
import warnings
from qiskit.circuit.library import RYGate, MCXGate
from sympy import fwht

# 1. Computing the WHT 

In [4]:
n=6
yk=[(2**(n)-1)/2]
for k in range(n):
    y=-2**(n-2)/2**k
    yk.append(y)
print(yk)
yk=yk/LA.norm(yk)
print(yk)

[31.5, -16.0, -8.0, -4.0, -2.0, -1.0, -0.5]
[ 0.86260912 -0.43815066 -0.21907533 -0.10953767 -0.05476883 -0.02738442
 -0.01369221]


# 2. Generate the circuit 

In [ ]:
n=6
n_qubits=n
backend_model=1
F=[]
L2=[]
backend_dict={0:'Statevector',
                  1:Aer.get_backend('aer_simulator'), 
                  2:AerSimulator.from_backend(FakeJakarta()) }#ibmq_belem

backend= backend_dict[backend_model]
circuit_aprx = QuantumCircuit(n,n)


for k0 in range(n+1):
    yk_approx=yk
    yk_approx = np.concatenate((yk_approx[0:k0+1], np.zeros(n-k0)))
   
    circuit_aprx = QuantumCircuit(n,n)
    circuit_exct = QuantumCircuit(n,n)
    
    correction=1
    correction_aprox=1
    ############################################## CIRCUIT for the exact state ###############################################
    for i in range(n):

        if i==0:

            circuit_aprx.ry(2*np.arcsin(yk_approx[1]),n-1)
            circuit_exct.ry(2*np.arcsin(yk[1]),n-1)
            
            correction=correction*np.cos(np.arcsin(yk[1]))
            correction_aprox=correction_aprox*np.cos(np.arcsin(yk_approx[1]))
            
        if i>0:

            for k in range(i):
                
                if i<k0:
                    circuit_aprx.x(n-1-k)      
                circuit_exct.x(n-1-k) 

            angle_exct=2*np.arcsin(yk[i+1]/correction) 
            if i<k0:
                angle_aprx=2*np.arcsin(yk_approx[i+1]/correction_aprox) 
            
            
            c3ry_exct = RYGate(angle_exct).control(i)
            if i<k0:
                c3ry_aprx = RYGate(angle_aprx).control(i)

            circuit_exct.append(c3ry_exct, list(range(n-i, n))+[n-i-1]) 
            
            if i<k0:
                circuit_aprx.append(c3ry_aprx, list(range(n-i, n))+[n-i-1]) 
            
            correction=correction*np.cos(angle_exct/2)
            if i<k0:
                correction_aprox=correction_aprox*np.cos(angle_aprx/2)
            
            for k in range(i):
                
                if i<k0:
                    circuit_aprx.x(n-1-k)      
                circuit_exct.x(n-1-k) 

    for i in range(n):
        circuit_aprx.h(i) 
        circuit_exct.h(i) 


    ############################################# Running the circuit ########################################

    tcirc_aprx = transpile(circuit_aprx, backend, optimization_level=1, basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'reset'])
    tcirc_exct = transpile(circuit_exct, backend, optimization_level=1, basis_gates=['id', 'rz', 'sx', 'x', 'cx', 'reset'])
  
    tcirc_aprx.save_statevector(label='state_before') 
    tcirc_aprx.save_density_matrix(label='rho_before') 
    tcirc_exct.save_statevector(label='state_before') 
    tcirc_exct.save_density_matrix(label='rho_before') 

    job_aprx=backend.run(tcirc_aprx, shots=8000)
    results_aprx = job_aprx.result()
  
    state_aprx=np.array(results_aprx.data(0)['state_before'])
    rho_aprx=np.array(results_aprx.data(0)['rho_before'])
 
    job_exct=backend.run(tcirc_exct, shots=8000)
    results_exct = job_exct.result()
    
    state_exct=np.array(results_exct.data(0)['state_before'])
    rho_exct=np.array(results_exct.data(0)['rho_before'])

    Fidelity=abs(np.array(results_aprx.data(0)['state_before'])@np.array(results_exct.data(0)['state_before']))**2
    L2_norm= np.sum(abs(abs(np.array(results_aprx.data(0)['state_before']))-abs(np.array(results_exct.data(0)['state_before']))))/2**(n/2)
    plt.plot(state_aprx) 
    F.append(Fidelity)
    L2.append(L2_norm)

In [10]:
plt.show()